In [33]:
import geopandas as gpd
import ee

ee.Initialize()

In [34]:
# Set filename
# gdf_filename = 'https://raw.githubusercontent.com/mapaction/assessment-resources/main/dataframe_1.geojson'
gdf_filename = 'https://raw.githubusercontent.com/mapaction/assessment-resources/main/dataframe_2.geojson'

# Load GeoDataFrame
gdf = gpd.read_file(gdf_filename)

In [35]:
gdf

,Name,geometry
0,A,"MULTIPOLYGON (((38.50207 32.83333, 38.48494 32..."
1,B,"MULTIPOLYGON (((35.95496 29.11667, 35.94378 29..."
2,C,"MULTIPOLYGON (((43.66047 36.58333, 43.65409 36..."
3,D,"MULTIPOLYGON (((44.24982 36.58351, 44.25000 36..."
4,E,"MULTIPOLYGON (((44.29176 36.60000, 44.29165 36..."


In [36]:
%%time

# Load WorldPop raster and restrict year
pop_collection = ee.ImageCollection("WorldPop/GP/100m/pop")
pop_year = pop_collection.filterMetadata('year', 'equals', 2020)

# Retrieve image scale
scale = pop_collection.first().projection().nominalScale().getInfo()

# Merge all images
pop_img = pop_collection.mosaic()

# Create empty ee_list
z_total = ee.List([])

# Loop over polygons
for i in range(len(gdf)):
    
    if gdf['geometry'].iloc[i].geom_type == 'Polygon':
        data_dict = [gdf['geometry'].iloc[i]]
    else:
        data_dict = list(gdf['geometry'].iloc[i].geoms)

    # Create list of polygons that constitutes the multipolygon
    gdf_pol = gpd.GeoDataFrame(
        {'geometry': data_dict},
        crs=gdf.crs
    )

    # Create feature collection out of GeoSeries
    polygons_fc = ee.FeatureCollection(gdf_pol.__geo_interface__)

    # Calculate zonal statistics
    zonal_statistics = pop_img.reduceRegions(
        reducer=ee.Reducer.sum(), 
        collection=polygons_fc, 
        scale=scale
    )

    # Create list with 'sum' values
    z_sum = zonal_statistics.aggregate_array('sum')
    
    # Calculate sum over the list
    z_pol = z_sum.reduce(ee.Reducer.sum())

    # Append result to ee_list
    z_total = z_total.add(z_pol)

# Print list
z_total.getInfo()

EEException: Request payload size exceeds the limit: 10485760 bytes.